In [73]:
import torch
import numpy as np
import re
from tqdm import tqdm
import pandas as pd

In [51]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [52]:
# Define parameters
num_particles = 10000
snr_dB = torch.arange(0, 31, 2, device=device)
v = 2.0
sigma = 1
beta = 1
iterations = 1
d_si = 30
d_it = 15
d_ir = 5
M = 100

In [53]:
# Define the objective function
def calculate_objective(params, snr_dB, v, sigma, beta, iterations, d_si, d_it, d_ir, M):
    alpha_t = params.item()
    alpha_r = 1-alpha_t

    capSTAR = torch.zeros(iterations, len(snr_dB), device=device)
    capOMA = torch.zeros(iterations, len(snr_dB), device=device)

    for jj in range(iterations):
        for ii, snr_db in enumerate(snr_dB):
            P = 10 ** (snr_db.item() / 10) * sigma
            Pr = 0.5 * P
            Pt = 0.5 * P

            # Channel coefficients for STAR-RIS
            hsi = torch.sqrt(torch.tensor(beta * d_si ** (-v))) * (torch.randn(2 * M) + 1j * torch.randn(2 * M))
            hit = torch.sqrt(torch.tensor(beta * d_it ** (-v))) * (torch.randn(2 * M) + 1j * torch.randn(2 * M))
            hir = torch.sqrt(torch.tensor(beta * d_ir ** (-v))) * (torch.randn(2 * M) + 1j * torch.randn(2 * M))

            hsi = hsi.to(device)
            hit = hit.to(device)
            hir = hir.to(device)

            # Phase coefficients for ES STAR-RIS
            theta_t = torch.linspace(0, 2 * np.pi, 2 * M, device=device)
            betatES = torch.rand(2 * M, device=device)
            PhitES = torch.diag(torch.sqrt(betatES) * torch.exp(1j * theta_t), 0)

            theta_r = torch.linspace(0, 2 * np.pi, 2 * M, device=device)
            betarES = 1 - betatES
            PhirES = torch.diag(torch.sqrt(betarES) * torch.exp(1j * theta_r), 0)

            # SNR calculation for STAR protocol
            N3 = P * alpha_t * (torch.abs(torch.matmul(torch.matmul(hit.t(), PhitES), hsi)) ** 2)
            N4 = P * alpha_r * (torch.abs(torch.matmul(torch.matmul(hir.t(), PhirES), hsi)) ** 2)
            D3 = P * alpha_t * (torch.abs(torch.matmul(torch.matmul(hit.t(), PhitES), hsi)) ** 2) + sigma
            gaamaSTAR = torch.min(N3 / D3, N4 / sigma)

            # SNR calculation for OMA protocol
            N3_OMA = Pt * (torch.abs(torch.matmul(torch.matmul(hit.t(), PhitES), hsi)) ** 2)
            N4_OMA = Pr * (torch.abs(torch.matmul(torch.matmul(hir.t(), PhirES), hsi)) ** 2)
            gaamaOMA = torch.min(N3_OMA / sigma, N4_OMA / sigma)

    return gaamaSTAR, gaamaOMA

In [54]:
# Sample text containing optimized parameters
text = """
Optimized Parameters:
Optimized alpha_t: [0.780085]
Optimized alpha_r: [0.21991497]

Optimized Parameters:
Optimized alpha_t: [0.7897605]
Optimized alpha_r: [0.21023953]

Optimized Parameters:
Optimized alpha_t: [0.78465325]
Optimized alpha_r: [0.21534675]

Optimized Parameters:
Optimized alpha_t: [0.7915346]
Optimized alpha_r: [0.2084654]

Optimized Parameters:
Optimized alpha_t: [0.76778233]
Optimized alpha_r: [0.23221767]

Optimized Parameters:
Optimized alpha_t: [0.787471]
Optimized alpha_r: [0.212529]

Optimized Parameters:
Optimized alpha_t: [0.78101265]
Optimized alpha_r: [0.21898735]

Optimized Parameters:
Optimized alpha_t: [0.77952933]
Optimized alpha_r: [0.22047067]

Optimized Parameters:
Optimized alpha_t: [0.77609164]
Optimized alpha_r: [0.22390836]

Optimized Parameters:
Optimized alpha_t: [0.7758649]
Optimized alpha_r: [0.2241351]

Optimized Parameters:
Optimized alpha_t: [0.7730439]
Optimized alpha_r: [0.22695613]

Optimized Parameters:
Optimized alpha_t: [0.7668829]
Optimized alpha_r: [0.2331171]

Optimized Parameters:
Optimized alpha_t: [0.7741247]
Optimized alpha_r: [0.22587532]

Optimized Parameters:
Optimized alpha_t: [0.7807999]
Optimized alpha_r: [0.21920007]

Optimized Parameters:
Optimized alpha_t: [0.7797874]
Optimized alpha_r: [0.22021258]

Optimized Parameters:
Optimized alpha_t: [0.77021563]
Optimized alpha_r: [0.22978437]

Optimized Parameters:
Optimized alpha_t: [0.78865063]
Optimized alpha_r: [0.21134937]

Optimized Parameters:
Optimized alpha_t: [0.7853358]
Optimized alpha_r: [0.21466422]

Optimized Parameters:
Optimized alpha_t: [0.78431654]
Optimized alpha_r: [0.21568346]

Optimized Parameters:
Optimized alpha_t: [0.77766037]
Optimized alpha_r: [0.22233963]
"""

In [55]:
# Extracting alpha_t values using regex
alpha_t_values = re.findall(r'Optimized alpha_t: \[(.*?)\]', text)

# Converting the extracted strings to floats
alpha_t_values = [float(value) for value in alpha_t_values]

print("Optimized alpha_t values:", alpha_t_values)

Optimized alpha_t values: [0.780085, 0.7897605, 0.78465325, 0.7915346, 0.76778233, 0.787471, 0.78101265, 0.77952933, 0.77609164, 0.7758649, 0.7730439, 0.7668829, 0.7741247, 0.7807999, 0.7797874, 0.77021563, 0.78865063, 0.7853358, 0.78431654, 0.77766037]


In [56]:
alpha_t_values = np.array(alpha_t_values)

In [57]:
min_ = -0.02
max_ = 0.01
random_vals = min_ + (np.random.randn(num_particles) * (max_ - min_))

In [58]:
alpha_t_values = random_vals + np.tile(alpha_t_values, num_particles//20)
alpha_r_values = 1 - alpha_t_values

In [59]:
len(alpha_t_values)

10000

In [60]:
# Optimization loop
snrs = []
for i in tqdm(range(num_particles)):
    snr = calculate_objective(torch.tensor(alpha_t_values[i]), snr_dB, v, 
                               sigma, beta, iterations, d_si, d_it, d_ir, M)
    snrs.append(snr)


100%|████████████████████████████████████████████████████████| 10000/10000 [05:12<00:00, 32.04it/s]


In [65]:
snrsdb = abs(10 * torch.log10(torch.tensor(snrs)))

In [88]:
snrsdb.shape

torch.Size([10000, 2])

In [90]:
snrsdb.numpy().shape

(10000, 2)

array([[ 0.75567628,  4.86194038,  4.9393487 ],
       [ 0.79939842,  2.90011024,  1.81468427],
       [ 0.79532093,  0.90295041,  4.34618568],
       ...,
       [ 0.74302235, 14.52858067, 11.63783455],
       [ 0.75988692,  2.54689956,  0.83557165],
       [ 0.73892132, 11.51142502, 12.88969421]])

In [99]:
data = np.concatenate([np.expand_dims(alpha_t_values, -1), 
                       np.expand_dims(alpha_r_values, -1), 
                       snrsdb.numpy()], axis=1)

In [101]:
df = pd.DataFrame(data,columns=["alpha_t", "alpha_r", "SNR_STAR", "SNR_OMA"])

In [103]:
df.to_csv("ece-snrs.csv", index=None)